In [17]:
from agents.structured_chat import StructuredChatAgent
from tools.base import Tool
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()

def search_func(args):
    input = args.get('input')
    return search.run(input)

search_tool = Tool(**{
    "name": "Current Search",
    "func": search_func,
    "description": "useful for when you need to answer questions about current events or the current state of the world",
    "args": {'input': {'type': 'string'}}
})


In [19]:
def company_query_func(args):
    name = args.get('name')
    members = {
        "James": "James is a member of esri company",
        "Jim": "Jim is a member of google company",
        "Jimmy": "Jimmy is a member of openai company",
        "Jimothy": "Jimothy is a member facebook company",
    }
    return f"{members[name]}"


company_query_tool = Tool(**{
    "name": "CompanyQuery",
    "func": company_query_func,
    "description": "useful for when you want to know someone's company",
    "args": {'name': {'type': 'string'}}
})

def company_info_func(args):
    name = args.get('name')
    members = {
        "esri": "esri is a geo related company",
        "google": "google is a search engine related company",
    }
    return f"{members[name]}"


company_info_tool = Tool(**{
    "name": "GetCompanyInfo",
    "func": company_info_func,
    "description": "useful for when you want to know a company's info",
    "args": {'name': {'type': 'string'}}
})

In [20]:
input = 'James belongs to which company? Introduce the company\'s information'
tools = [company_query_tool, company_info_tool]
agent = StructuredChatAgent(tools, verbose=True)

In [21]:
agent.run(input=input)

> Start: 
James belongs to which company? Introduce the company's information

Thought: I need to use the CompanyQuery tool to find out which company James belongs to. Once I have the company name, I can use the GetCompanyInfo tool to retrieve the company's information.

Action:
```
{
  "action": "CompanyQuery",
  "action_input": {
    "name": "James"
  }
}
```
Observation: James is a member of esri company

Thought:Now that I know James belongs to the "esri" company, I can use the GetCompanyInfo tool to retrieve the company's information.

Action:
```
{
  "action": "GetCompanyInfo",
  "action_input": {
    "name": "esri"
  }
}
```
Observation: esri is a geo related company

Thought:Based on the information I found, James belongs to the company "esri". Esri is a geo-related company.
Observation: Based on the information I found, James belongs to the company "esri". Esri is a geo-related company.



'Based on the information I found, James belongs to the company "esri". Esri is a geo-related company.'